[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/cookbook/blob/main/gen-ai/google-ai/gemini-2/web-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/cookbook/blob/main/gen-ai/google-ai/gemini-2/web-search.ipynb)

# Gemini 2.0 Flash Web Search + Citations

## Setup Instructions

This notebook has been tested both locally with Python `3.12.7` and in Google Colab with Python `3.10.12`.

To run locally, please refer to the [setup instructions with `uv` here](https://github.com/aurelio-labs/agents-course/blob/main/local-setup.md). To run in Google Colab, simply run all cells in the notebook.

## Setup with Google AI

To begin, we will need a Google API key. For that, you can setup an account in [Google AI Studio](https://aistudio.google.com). To initialize and begin using Gemini 2 that is all we need, but later we will need to upgrade our plan to get access to the Google Search tool.

After you have your account and API key, we initialize our [`google.genai` client](https://github.com/googleapis/python-genai):

In [1]:
import os
from getpass import getpass
from google import genai

# pass your API key here
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass(
    "Enter Google API Key: "
)
# initialize our client
client = genai.Client()

To generate some text with Gemini we call the `generate_content` method, Gemini does tend to respond with markdown, and is pretty verbose, so we'll use IPython's `Markdown` to display the output.

In [2]:
from IPython.display import Markdown

model_id = "gemini-2.0-flash-exp"

response = client.models.generate_content(
    model=model_id, contents='What is Gemini?'
)
Markdown(response.text)

Gemini is a family of large language models (LLMs) developed by Google. It's designed to be multimodal, meaning it can understand and process information across different types of data, including text, code, audio, images, and video. 

Here's a breakdown of key aspects of Gemini:

**Key Features:**

* **Multimodal Capabilities:** This is Gemini's biggest differentiator. Unlike many previous LLMs that primarily focused on text, Gemini can work with a wider range of inputs, making it more versatile and powerful.
* **Advanced Reasoning and Problem Solving:** Gemini is designed to tackle complex tasks, including those requiring logical reasoning, mathematical calculations, and even abstract thinking.
* **Code Generation:** It can write code in various programming languages, making it a valuable tool for developers.
* **Image and Video Understanding:** Gemini can analyze images and videos, identify objects, and understand the content being presented.
* **Natural Language Understanding and Generation:** Like other LLMs, Gemini excels at understanding and generating human-like text. This includes tasks like translation, summarization, creative writing, and question answering.
* **Scalability:** Gemini models come in different sizes (Ultra, Pro, and Nano), allowing them to be deployed on a wide range of devices, from data centers to mobile phones.
* **Integration with Google Products:** Gemini is being integrated into various Google products, such as Google Search, Bard, Workspace apps, and Android.

**Gemini Models:**

* **Gemini Ultra:** The most powerful model, designed for complex and demanding tasks. It is intended for use in data centers and cloud environments.
* **Gemini Pro:** A balanced model that offers strong performance across a wide range of tasks and is well-suited for general-purpose applications.
* **Gemini Nano:** The smallest and most efficient model, designed for on-device processing on mobile phones and other resource-constrained devices.

**How it's Different from Previous Models:**

* **Multimodality:** This is the biggest difference. Prior LLMs were largely text-based. Gemini's ability to handle multiple modalities is a significant advancement.
* **Advanced Capabilities:** Gemini has been shown to outperform previous models on various benchmarks, indicating its superior reasoning and problem-solving skills.
* **Scalability:** The availability of different model sizes makes Gemini more accessible for a wider range of use cases.

**Potential Uses:**

Gemini has the potential to be used in a wide variety of applications, including:

* **Search:** Enhancing search results with more accurate and comprehensive information.
* **Content Creation:** Generating text, code, images, and videos.
* **Translation:** Providing more accurate and natural-sounding translations.
* **Personalized Experiences:** Tailoring content and services to individual needs.
* **Scientific Research:** Accelerating discoveries by analyzing complex data.
* **Education:** Creating new learning tools and personalized educational experiences.
* **Healthcare:** Assisting doctors with diagnosis and treatment plans.

**In Summary:**

Gemini is a cutting-edge family of large language models that represents a significant leap forward in AI technology. Its multimodal capabilities, advanced reasoning, and scalability make it a powerful tool with the potential to impact many different areas of our lives. As the technology continues to evolve, we can expect even more exciting applications and advancements based on the Gemini architecture.


We got a pretty descriptive response. Now let's take a look at tool use in Gemini, starting with the `GoogleSearch` tool.

_Note: to use this tool, you **must** upgrade your plan, you can do that by navigating to your Google AI studio settings and clicking the **API Plan Information** link. For the purposes of this notebook, you should be covered by their free allowance but in any case you must have billing enabled._

In [3]:
from google.genai.types import Tool, GoogleSearch

search_tool = Tool(google_search=GoogleSearch())

To use the tool, we must create a [`GenerateContentConfig` object](https://github.com/googleapis/python-genai/blob/cf3c476ea300acc09b514d23b5f1f08c05efce60/google/genai/types.py#L1404). This object is used to configure input into Gemini, this can include `system_instructions`, `temperature`, `max_output_tokens`, our `response_modalities`, and _very importantly_, our `tools`.

Most of these parameters are optional and the _defaults_ seem to depend on the model being used although it isn't clear what the default parameters are for Gemini 2.0 Flash. Nonetheless, for parameters whose defaults are more easily guessable we have included them in our `config` object definition below.

In [4]:
from google.genai.types import GenerateContentConfig

config = GenerateContentConfig(
    system_instruction=(
        "You are a helpful assistant that provides up to date information "
        "to help the user in their research."
    ),
    tools=[search_tool],
    response_modalities=["TEXT"],
    temperature=0.0,  # likely default
    candidate_count=1,  # likely default
    # following are some other parameters, all optional
    #top_p,
    #top_k,
    #max_output_tokens,
    #max_tokens,
    #stop_sequences,
    #safety_settings,
)
config

GenerateContentConfig(system_instruction='You are a helpful assistant that provides up to date information to help the user in their research.', temperature=0.0, top_p=None, top_k=None, candidate_count=1, max_output_tokens=None, stop_sequences=None, response_logprobs=None, logprobs=None, presence_penalty=None, frequency_penalty=None, seed=None, response_mime_type=None, response_schema=None, routing_config=None, safety_settings=None, tools=[Tool(function_declarations=None, retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, code_execution=None)], tool_config=None, cached_content=None, response_modalities=['TEXT'], media_resolution=None, speech_config=None, automatic_function_calling=None)

Now we have our `config` setup, we can go ahead and ask Gemini about the latest news in AI, by default Gemini would not be able to answer this question, but we can get up to date information by augmenting Gemini's knowledge with the `GoogleSearch` tool.

In [5]:
response = client.models.generate_content(
    model=model_id,
    contents="Tell me the latest news in AI",
    config=config
)
Markdown(response.text)

Here's a rundown of the latest news in AI:

**AI Model Development & Advancements:**

*   **Google's New Generative AI Models:** Google Cloud has launched two new generative AI models, Veo and Imagen 3, on its Vertex AI platform. This comes amid reports of significant revenue growth for enterprises using generative AI.
*   **Teaching Robots Limits:** MIT researchers have developed a method called "PRoC3S" that helps large language models (LLMs) create safe action plans for robots by testing each step in a simulation. This could help robots complete more complex tasks in the future.
*   **Reducing Bias in AI:** Researchers have created a new technique to identify and remove training examples that contribute to failures in machine learning models, which helps reduce bias while maintaining or improving accuracy.
*   **AI Explanations in Plain Language:** Researchers are using LLMs to convert machine-learning explanations into easy-to-understand narratives. This could help users make better decisions about when to trust an AI model.

**AI in Specific Fields:**

*   **AI and Mental Health:** There is growing discussion about the potential of AI in mental health, with experts exploring its possibilities.
*   **AI in Healthcare:** AI is changing how doctors access medical knowledge. There is also a call for regulation of AI models and non-AI algorithms in healthcare.
*   **AI in Media:** AI video generation is transforming media, with new tools like Sora emerging. This raises questions about the strengths and challenges of AI-generated content.

**Other Notable AI News:**

*   **AI Political Bias:** Research from MIT has found that some language reward models exhibit political bias, even when trained on factual data.
*   **AI Deepfakes:** There is increasing discussion on how to spot AI deepfake videos in an era of digital deception.
*   **AI Partner Programs:** Alibaba Cloud is overhauling its AI partner ecosystem with new initiatives, including an AI partner accelerator program.
*   **AI in Robotics:** Daniela Rus, director of MIT CSAIL, has won the John Scott Award for her robotics research.

**General Trends:**

*   **AI Regulation:** There is an ongoing discussion about the need for regulation of AI, particularly in sensitive fields like healthcare.
*   **AI Learning:** There are many free online courses available for those who want to learn about AI in 2025.

This information is based on the latest news articles available as of today, December 14, 2024.


We can look at the `grounding_metadata` to see what sources were used to generate the response.

In [6]:
response.candidates[0].grounding_metadata.__dict__

{'grounding_chunks': [GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='artificialintelligence-news.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==')),
  GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='mit.edu', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=')),
  GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='forbes.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcSGcTcJ4fsucHQfs_GXQ2-yE8eORAiz-lPxOcdt7Z-Ijr1yizEt2vHBaJo_Xch79fUka-8xfRSK2G69kJYZmjCJajX8ofcsDr4g6R7f9P2aAdL2iZkrmCQtuWsRd25xD1sI00OC2FdExhskRPY='))],
 'grounding_supports': [GroundingSupport(confidence_s

Each source used is collected inside a `GroundingChunk` object. Inside that we can access the `web` property which contains a `GroundingChunkWeb` containing the source website `title` and `uri`, which we can use to build a list of sources:

In [7]:
link_str = ""
for i, chunk in enumerate(response.candidates[0].grounding_metadata.grounding_chunks):
    title = chunk.web.title
    link = chunk.web.uri
    link_str += f"[{i+1}] [{title}]({link})\n\n"
Markdown(link_str)

[1] [artificialintelligence-news.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)

[2] [mit.edu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)

[3] [forbes.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcSGcTcJ4fsucHQfs_GXQ2-yE8eORAiz-lPxOcdt7Z-Ijr1yizEt2vHBaJo_Xch79fUka-8xfRSK2G69kJYZmjCJajX8ofcsDr4g6R7f9P2aAdL2iZkrmCQtuWsRd25xD1sI00OC2FdExhskRPY=)



That's great, but we can go a little deeper. Inside `response.candidates[0].grounding_metadata` we also have a `grounding_support` property containing a list of `GroundingSupport` objects, each of those mapping to the `GroundingChunk` objects that we already parsed above. We'll combine this information with our `title` and `link` data to create a list of `Citation` objects.

In [8]:
from pydantic import BaseModel

class Citation(BaseModel):
    title: str
    score: float
    start_index: int
    end_index: int
    chunk_index: int
    link: str

    def get_link(self):
        return f"_[[{self.chunk_index+1}]({self.link})]_"
    
    def count_chars(self):
        return len(self.get_link())

In [9]:
citations = []
for support in response.candidates[0].grounding_metadata.grounding_supports:
    chunk_index = support.grounding_chunk_indices[0]
    citations.append(
        Citation(
            title=response.candidates[0].grounding_metadata.grounding_chunks[chunk_index].web.title,
            score=support.confidence_scores[0],
            start_index=support.segment.start_index,
            end_index=support.segment.end_index,
            chunk_index=chunk_index,
            link=response.candidates[0].grounding_metadata.grounding_chunks[chunk_index].web.uri,
        )
    )

In [10]:
# sort citations by start_index
citations.sort(key=lambda x: x.start_index)
citations

[Citation(title='artificialintelligence-news.com', score=0.8782686, start_index=86, end_index=229, chunk_index=0, link='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw=='),
 Citation(title='artificialintelligence-news.com', score=0.8888342, start_index=230, end_index=320, chunk_index=0, link='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw=='),
 Citation(title='mit.edu', score=0.73191243, start_index=321, end_index=522, chunk_index=1, link='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI='),
 Citation(title='mit.edu', score

We can get our citation links like so:

In [11]:
citations[0].get_link()

'_[[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)]_'

With all of this information, we can go back to our original `response.text` and add these into our response.

In [12]:
final_response = response.text
offset = 0
for citation in citations:
    final_response = (
        final_response[:citation.end_index+offset] +
        citation.get_link() +
        final_response[citation.end_index+offset:]
    )
    offset += citation.count_chars()
Markdown(final_response)

Here's a rundown of the latest news in AI:

**AI Model Development & Advancements:**

*   **Google's New Generative AI Models:** Google Cloud has launched two new generative AI models, Veo and Imagen 3, on its Vertex AI platform._[[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)]_ This comes amid reports of significant revenue growth for enterprises using generative AI._[[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)]_
*   **Teaching Robots Limits:** MIT researchers have developed a method called "PRoC3S" that helps large language models (LLMs) create safe action plans for robots by testing each step in a simulation._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_ This could help robots complete more complex tasks in the future._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **Reducing Bias in AI:** Researchers have created a new technique to identify and remove training examples that contribute to failures in machine learning models, which helps reduce bias while maintaining or improving accuracy._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **AI Explanations in Plain Language:** Researchers are using LLMs to convert machine-learning explanations into easy-to-understand narratives._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_ This could help users make better decisions about when to trust an AI model._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_

**AI in Specific Fields:**

*   **AI and Mental Health:** There is growing discussion about the potential of AI in mental health, with experts exploring its possibilities.
*   **AI in Healthcare:** AI is changing how doctors access medical knowledge. There is also a call for regulation of AI models and non-AI algorithms in healthcare._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **AI in Media:** AI video generation is transforming media, with new tools like Sora emerging. This raises questions about the strengths and challenges of AI-generated content.

**Other Notable AI News:**

*   **AI Political Bias:** Research from MIT has found that some language reward models exhibit political bias, even when trained on factual data._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **AI Deepfakes:** There is increasing discussion on how to spot AI deepfake videos in an era of digital deception.
*   **AI Partner Programs:** Alibaba Cloud is overhauling its AI partner ecosystem with new initiatives, including an AI partner accelerator program._[[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)]_
*   **AI in Robotics:** Daniela Rus, director of MIT CSAIL, has won the John Scott Award for her robotics research._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_

**General Trends:**

*   **AI Regulation:** There is an ongoing discussion about the need for regulation of AI, particularly in sensitive fields like healthcare.
*   **AI Learning:** There are many free online courses available for those who want to learn about AI in 2025._[[3](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcSGcTcJ4fsucHQfs_GXQ2-yE8eORAiz-lPxOcdt7Z-Ijr1yizEt2vHBaJo_Xch79fUka-8xfRSK2G69kJYZmjCJajX8ofcsDr4g6R7f9P2aAdL2iZkrmCQtuWsRd25xD1sI00OC2FdExhskRPY=)]_

This information is based on the latest news articles available as of today, December 14, 2024.


We now have functional inline citations inside our response, let's finish by adding a final block containing the more detailed citations.

In [13]:
final_response = f"{final_response}\n\n**Citations**\n\n{link_str}"
Markdown(final_response)

Here's a rundown of the latest news in AI:

**AI Model Development & Advancements:**

*   **Google's New Generative AI Models:** Google Cloud has launched two new generative AI models, Veo and Imagen 3, on its Vertex AI platform._[[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)]_ This comes amid reports of significant revenue growth for enterprises using generative AI._[[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)]_
*   **Teaching Robots Limits:** MIT researchers have developed a method called "PRoC3S" that helps large language models (LLMs) create safe action plans for robots by testing each step in a simulation._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_ This could help robots complete more complex tasks in the future._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **Reducing Bias in AI:** Researchers have created a new technique to identify and remove training examples that contribute to failures in machine learning models, which helps reduce bias while maintaining or improving accuracy._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **AI Explanations in Plain Language:** Researchers are using LLMs to convert machine-learning explanations into easy-to-understand narratives._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_ This could help users make better decisions about when to trust an AI model._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_

**AI in Specific Fields:**

*   **AI and Mental Health:** There is growing discussion about the potential of AI in mental health, with experts exploring its possibilities.
*   **AI in Healthcare:** AI is changing how doctors access medical knowledge. There is also a call for regulation of AI models and non-AI algorithms in healthcare._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **AI in Media:** AI video generation is transforming media, with new tools like Sora emerging. This raises questions about the strengths and challenges of AI-generated content.

**Other Notable AI News:**

*   **AI Political Bias:** Research from MIT has found that some language reward models exhibit political bias, even when trained on factual data._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_
*   **AI Deepfakes:** There is increasing discussion on how to spot AI deepfake videos in an era of digital deception.
*   **AI Partner Programs:** Alibaba Cloud is overhauling its AI partner ecosystem with new initiatives, including an AI partner accelerator program._[[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)]_
*   **AI in Robotics:** Daniela Rus, director of MIT CSAIL, has won the John Scott Award for her robotics research._[[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)]_

**General Trends:**

*   **AI Regulation:** There is an ongoing discussion about the need for regulation of AI, particularly in sensitive fields like healthcare.
*   **AI Learning:** There are many free online courses available for those who want to learn about AI in 2025._[[3](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcSGcTcJ4fsucHQfs_GXQ2-yE8eORAiz-lPxOcdt7Z-Ijr1yizEt2vHBaJo_Xch79fUka-8xfRSK2G69kJYZmjCJajX8ofcsDr4g6R7f9P2aAdL2iZkrmCQtuWsRd25xD1sI00OC2FdExhskRPY=)]_

This information is based on the latest news articles available as of today, December 14, 2024.


**Citations**

[1] [artificialintelligence-news.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcRQ5GdEmPgNzz3r4AaXQCtleWqP29sDHuwddXo_M992yvPTKTJZ8XapssxVUUxcWNLhCpAUNm0XMrlfmJ53DmO7tQDUmu3iOwOuo8vb08TRdaXL8jSguBgqUdjGduo3KdQTQw==)

[2] [mit.edu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQwqsNU9baZEBILLWnA5mDuqvUSEfmb_E12Z8K6sa0MlOs-oOXZptSmFPVld5veeEV-cp9HsYpnrBzHGYLmtotIJl-m_rI2LQxpGEqn4Ywish1WATGSkv1RCCFn0vqfr4NlqQcVwN-kobI=)

[3] [forbes.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcSGcTcJ4fsucHQfs_GXQ2-yE8eORAiz-lPxOcdt7Z-Ijr1yizEt2vHBaJo_Xch79fUka-8xfRSK2G69kJYZmjCJajX8ofcsDr4g6R7f9P2aAdL2iZkrmCQtuWsRd25xD1sI00OC2FdExhskRPY=)



---